 # Thyroid Disease Binary Classification

In [ ]:
!pip install -r requirements.txt --quiet

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

tdf = pd.read_csv("data/MLDataR_thyroid_class.csv")

In [ ]:
tdf.describe()

In [ ]:
tdf.columns

## Exploratory Data Analysis  {EDA}

In [ ]:
from modelviz.histogram import plot_feature_histograms
plot_feature_histograms(tdf, library='matplotlib', 
                        exclude_bin_encode=True)

## Visualize missing values

In [ ]:
from modelviz.missvals import plot_missing_values_heatmap

In [ ]:
plot_missing_values_heatmap(tdf)

## Correlation matrix

In [ ]:
from modelviz.relationships import plot_correlation_matrix

In [ ]:
plot_correlation_matrix(tdf, max_columns=18)

## Class Distribution

In [ ]:
pd.Series(tdf['ThyroidClass']).value_counts().plot(
    kind='bar', title='Class Distribution')

# Work on missing value volumes


In [ ]:
def missing_values_summarizer(df:pd.DataFrame, 
                              drop_cols:bool=False, 
                              drop_threshold:float=0.1, 
                              verbose=False):

    missing_counts = df.isnull().sum()
    non_missing_counts = df.notnull().sum()
    total_vol = len(df)
    missing_proportions = missing_counts / len(df)
    non_missing_proportions = non_missing_counts / len(df)

    props_df = pd.DataFrame({
        "feature": df.columns,
        "missing_count": missing_counts,
        "non_missing_count": non_missing_counts,
        "total_vol": total_vol,
        "prop_missing": missing_proportions,
        "prop_non_missing": non_missing_proportions,
        'data_type': df.dtypes.values
    })
    props_df.reset_index(inplace=True, drop=True)
    
    if drop_cols:
        cols_to_drop=props_df[props_df['prop_missing'] > drop_threshold]['feature'].tolist()
        df_resized = df.drop(columns=cols_to_drop)
        if verbose:
            print(f'Dropping columns: {cols_to_drop}')
    else:
        df_resized = df.copy()
    
    return props_df, df_resized


In [ ]:
props_df, res_df = missing_values_summarizer(tdf, 
                                             drop_cols=True, 
                                             drop_threshold=0.5)

In [ ]:
props_df

In [ ]:
res_df.columns

In [ ]:
res_df.drop(columns=['TSH_reading'], 
            axis=1, 
            inplace=True)

## Remove Columns with Zero Variance

In [ ]:
variances = res_df.var(numeric_only=True)
zero_variance_columns = variances[variances == 0].index.tolist()
print(zero_variance_columns)
res_df.drop(columns=zero_variance_columns, inplace=True)

## Split data

In [ ]:
from sklearn.model_selection import train_test_split
X = res_df.drop(columns='ThyroidClass', axis=1)
y = pd.Series(res_df['ThyroidClass'])


In [ ]:
label_mapping = {
    'negative': 0,
    'sick': 1
}

y_mapped = y.map(label_mapping)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y_mapped, 
                                                    stratify=y_mapped, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [ ]:
def get_split_sizes(X_train, X_test):
    X_train_size = len(X_train)
    X_train_cols = len(X_train.columns)

    X_test_size = len(X_test)
    X_test_cols = len(X_train.columns)

    if X_train_cols != X_test_cols:
        assert ValueError('Columns in both DataFrames should match, as X_train has: {X_train_cols} columns and X_test has: {X_test_cols} columns')

    print(
        f"Train and testing set statistics\n\n"
        f"X_train size: {X_train_size} | X_train columns: {X_train_cols}\n"
        f"X_test size: {X_test_size} | X_test columns: {X_test_cols}")
    

In [ ]:
get_split_sizes(X_train, X_test)

## One Hot encode `ref_src` columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def encode_categorical_columns(X_train, X_test, categorical_columns, drop='first'):
    encoder = OneHotEncoder(sparse_output=False, 
                            handle_unknown='ignore', 
                            drop=drop)
    encoder.fit(X_train[categorical_columns])
    X_train_encoded_array = encoder.transform(X_train[categorical_columns]).astype(float)
    X_test_encoded_array = encoder.transform(X_test[categorical_columns]).astype(float)
    encoded_feature_names = encoder.get_feature_names_out(categorical_columns)

    X_train_encoded_df = pd.DataFrame(X_train_encoded_array, 
                                      columns=encoded_feature_names, 
                                      index=X_train.index)
    X_test_encoded_df = pd.DataFrame(X_test_encoded_array, 
                                     columns=encoded_feature_names, 
                                     index=X_test.index)
    print("Encoded Data Example (Train):")
    print(X_train_encoded_df.head())

    X_train = X_train.drop(columns=categorical_columns)
    X_test = X_test.drop(columns=categorical_columns)
  
    X_train_encoded = pd.concat([X_train.reset_index(drop=True), 
                                 X_train_encoded_df.reset_index(drop=True)], axis=1)
    X_test_encoded = pd.concat([X_test.reset_index(drop=True), 
                                X_test_encoded_df.reset_index(drop=True)], axis=1)
    return X_train_encoded, X_test_encoded


In [ ]:
X_train_ohenc, X_test_ohenc = encode_categorical_columns(
    X_train=X_train, 
    X_test=X_test, 
    categorical_columns=['ref_src'])

## Using MICE to impute missing values

In [ ]:
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer, SimpleImputer

In [ ]:
use_simple = False
if use_simple:
    imputer = SimpleImputer()
else:
    imputer = IterativeImputer(random_state=42)
imputed_data = imputer.fit_transform(X_train_ohenc)

In [ ]:
# Transform both training and testing data
X_train_imputed = imputer.transform(X_train_ohenc)
X_test_imputed = imputer.transform(X_test_ohenc)

In [ ]:
import numpy as np
if np.isnan(X_train_imputed).any() | np.isnan(X_test_imputed).any():
    print('Null values remain')

## Over and under sampling techniques 

| **Type**         | **Python Package**      | **Description**                                                                 |
|-------------------|-------------------------|---------------------------------------------------------------------------------|
| **SMOTE**        | `imbalanced-learn`      | Synthetic Minority Oversampling Technique. Generates synthetic samples by interpolating between existing minority samples. Suitable for numeric datasets. |
| **ROS (Random Over-Sampling)** | `imbalanced-learn`      | Replicates samples from the minority class to balance the dataset. Simple but can lead to overfitting. |
| **ADASYN**       | `imbalanced-learn`      | Adaptive Synthetic Sampling. Focuses on generating synthetic samples in regions where the minority class is sparse. Variant of SMOTE. |
| **ROSE (Random Over-Sampling Examples)** | No direct Python implementation (alternatives: custom implementation, ROS, ADASYN) | Adds noise to oversampled data points to create diverse synthetic examples. Native to R; Python alternatives include ROS and ADASYN. |
| **SMOTEENN**     | `imbalanced-learn`      | Combination of SMOTE (oversampling) and Edited Nearest Neighbors (undersampling). Balances the dataset and removes noisy samples. |
| **SMOTETomek**   | `imbalanced-learn`      | Combines SMOTE with Tomek Links (undersampling technique). Removes borderline samples after oversampling. |
| **Random Under-Sampling** | `imbalanced-learn`      | Reduces the dataset by removing samples from the majority class to balance the dataset. Can lead to loss of important information. |
| **Cluster-Based Over-Sampling** | Custom implementation required | Uses clustering (e.g., k-means) to create synthetic samples by identifying dense regions in the feature space. Experimental technique. |
| **Class Weight Adjustment** | `scikit-learn`          | Adjusts model training weights to penalize misclassification of the minority class more heavily. Does not change the dataset but impacts model training. |


## Normalization - Scaling our data

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
robust_scale = RobustScaler()

In [ ]:
X_train_scaled = robust_scale.fit_transform(X_train_imputed)

In [ ]:
X_test_scaled = robust_scale.transform(X_test_imputed)

In [ ]:
assert not np.isnan(X_train_scaled).any(),"NaN values found in X_train_pp."
assert not np.isnan(X_test_scaled).any(), "NaN values found in X_test_pp."

## Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
k = 15  
selector = SelectKBest(score_func=f_classif, k=k)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)
selected_features = selector.get_feature_names_out()

## Modeling time

In [ ]:
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import lightgbm as lgb

In [ ]:
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
print(scale_pos_weight)

In [ ]:
models = {
    "logistic_regression": LogisticRegression(max_iter=2000, 
                                              random_state=42, 
                                              class_weight='balanced'),
    "svc": SVC(kernel='rbf', probability=True, 
               random_state=42, 
               class_weight='balanced'),

    "cat_boost": CatBoostClassifier(iterations=500, 
                                    depth=6, 
                                    learning_rate=0.1, 
                                    class_weights=[1, 10], 
                                    verbose=0),

    "xgboost_scaled": XGBClassifier(n_estimators=500, 
                                    max_depth=6, 
                                    learning_rate=0.1, 
                                    scale_pos_weight=scale_pos_weight, 
                                    random_state=42),

    "xgboost_unscaled": XGBClassifier(n_estimators=500, 
                                      max_dept=6, 
                                      learning_rate=0.1, 
                                      random_state=42),

    "LGBM": lgb.LGBMClassifier(n_estimators=500,
                               learning_rate=0.1,
                               class_weight='balanced',
                               random_state=42, 
                               max_depth=6)
}

In [ ]:
# Define StratifiedKFold
from sklearn.model_selection import StratifiedKFold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, 
                      shuffle=True, 
                      random_state=42)

In [ ]:
fitted_models = {}

In [ ]:
X_train_final = X_train_selected
X_test_final = X_test_selected
y_train_final = y_train

In [ ]:
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
    confusion_matrix,
    roc_auc_score
)

def compute_metrics(y_true, y_pred, y_pred_proba):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    metrics = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy,
        'roc_auc': roc_auc,
        'confusion_matrix': cm
    }
    return metrics


## Confusion matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_simple_confusion_matrix(cm, classes, 
                                 model_name, normalize=False, 
                                 cmap='Blues'):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        fmt = '.2f'
        title = f'Normalized Confusion Matrix for {model_name}'
    else:
        fmt = 'd'
        title = f'Confusion Matrix for {model_name}'

    plt.figure(figsize=(6,6))
    sns.heatmap(cm, annot=True, fmt=fmt, cmap=cmap, cbar=False,
                xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()


In [ ]:
from modelviz.confusion_matrix import plot_confusion_matrix

In [ ]:
from sklearn.model_selection import cross_val_score
from imblearn.pipeline import Pipeline
from sklearn.base import clone
from imblearn.combine import SMOTEENN

fitted_models = {}
model_scores = {}
threshold_results = {}
EVAL_METRIC = 'recall' 
ADJUSTED_THRESHOLD = 0.5


In [ ]:
%%capture
for model_name, model in models.items():
    print(f"Evaluating {model_name}...")
    
    # Perform cross-validation with SMOTEENN included in the pipeline
    pipeline = Pipeline([
        ('smoteenn', SMOTEENN(random_state=42, sampling_strategy=0.5)),
        ('model', model)
    ])
    cv_scores = cross_val_score(pipeline, X_train_final, 
        y_train_final, cv=skf, scoring=EVAL_METRIC)
    
    # Output cross-validation scores
    mean_score = cv_scores.mean()
    std_score = cv_scores.std()
    model_scores[model_name] = {'mean_score': mean_score, 
                                'std_score': std_score}
    print(f"{model_name} Mean {EVAL_METRIC} Score: {mean_score:.4f} (+/- {std_score:.4f})\n")
    
    # Apply SMOTEENN to the entire training data
    smoteenn = SMOTEENN(random_state=42, 
                        sampling_strategy=0.5)
    X_resampled, y_resampled = smoteenn.fit_resample(X_train_final, y_train_final)
    
    # Clone the model to avoid reusing the one from cross-validation
    model_clone = clone(model)
    
    # Fit the model on the resampled training data
    model_clone.fit(X_resampled, y_resampled)
    fitted_models[model_name] = model_clone
    
    # Predict probabilities on the test data (without resampling)
    y_pred_proba = model_clone.predict_proba(X_test_final)[:, 1]
    
    # Adjust predictions using the new threshold
    y_pred_adjusted = (y_pred_proba >= ADJUSTED_THRESHOLD).astype(int)
    
    # Evaluate metrics
    metrics = compute_metrics(y_test, y_pred_adjusted, y_pred_proba)
    
    # Store metrics
    threshold_results[model_name] = metrics


In [ ]:
threshold_results['cat_boost']

In [ ]:
from modelviz.roc import plot_roc_curve_with_youdens_thresholds
from sklearn.metrics import roc_curve

In [ ]:
def evaluate_model(model_name, model, X_test, y_test, 
                   adjusted_threshold=0.5, 
                   classes=['Negative', 'Sick'], 
                   calculate_youdens=False, plot_simple_cm=False):
    print(f"Evaluating {model_name} on test data...")

    # Predict probabilities and apply adjusted threshold
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred_adjusted = (y_pred_proba >= adjusted_threshold).astype(int)
    adjusted_metrics = compute_metrics(y_test, y_pred_adjusted, y_pred_proba)

    # Compute ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(y_test, 
                                     y_pred_proba)

    evaluation_results = {
        'fitted_model': model,
        'adjusted_threshold': adjusted_threshold,
        'metrics_at_adjusted_threshold': adjusted_metrics,
        'roc_curve': (fpr, tpr, thresholds),
    }

    if calculate_youdens:
        # Find optimal threshold using Youden's J statistic
        youdens_j = tpr - fpr
        optimal_idx = np.argmax(youdens_j)
        optimal_threshold = thresholds[optimal_idx]
        max_youdens_j = youdens_j[optimal_idx]
        print(f"Maximum Youden's J for {model_name}: {max_youdens_j:.4f} at threshold {optimal_threshold:.4f}\n")
        # Apply optimal threshold and compute metrics
        y_pred_youden = (y_pred_proba >= optimal_threshold).astype(int)
        youden_metrics = compute_metrics(
            y_test, 
            y_pred_youden, 
            y_pred_proba)

        # Update evaluation results with Youden's metrics
        evaluation_results.update({
            'optimal_threshold': optimal_threshold,
            'metrics_at_optimal_threshold': youden_metrics,
            'max_youdens_j': max_youdens_j
        })

        # Plot Confusion Matrix for Youden's threshold
        plot_confusion_matrix(
            youden_metrics['confusion_matrix'], 
            classes=classes,
            model_name=f"{model_name} (Youden's J Threshold)",
            table_fontsize=8)

    if plot_simple_cm:
        plot_simple_confusion_matrix(
            adjusted_metrics['confusion_matrix'], 
            classes=classes,
            model_name=f"{model_name} (Adjusted Threshold)")
        
    else: 
        # Plot Confusion Matrix for Adjusted Threshold
        plot_confusion_matrix(
            adjusted_metrics['confusion_matrix'], classes=classes,
            model_name=f"{model_name} (Adjusted Threshold)",
            table_fontsize=8)
    # Plot ROC Curve with thresholds
    plot_roc_curve_with_youdens_thresholds(
        fpr, tpr, 
        thresholds, 
        adjusted_metrics['roc_auc'],
        model_name, 
        adjusted_threshold, 
        optimal_threshold if calculate_youdens else None,
        xlabel='False Positive Rate (1-FPR)',
        ylabel='True Positive Rate (TPR)')

    return evaluation_results


In [ ]:
# Define your adjusted threshold
ADJUSTED_THRESHOLD = 0.5  
# Dictionary to store all evaluation results
all_evaluation_results = {}

for model_name, model in fitted_models.items():

    evaluation_results = evaluate_model(
        model_name=model_name,
        model=model,
        X_test=X_test_final,
        y_test=y_test,
        adjusted_threshold=ADJUSTED_THRESHOLD,
        calculate_youdens=True, 
        plot_simple_cm=False
    )
    
    all_evaluation_results[model_name] = evaluation_results


In [ ]:
all_evaluation_results